# Daily Star News Scraping

Here news articles from 'The Daily Star' newspaper of Bangladesh that are about road accident Bangladesh were scraped. The Daily Star website's search bar was used to find the related news articles. The search string used was 'Road accident Bangladesh'.

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import warnings

try:
    import dateparser
except:
    !pip install dateparser
    import dateparser

!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)
wd = webdriver.Chrome('chromedriver',options=chrome_options)

     |████████████████████████████████| 286kB 5.1MB/s 
     |████████████████████████████████| 911kB 5.2MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http:/

In [3]:
urls = ["https://www.thedailystar.net/google/search#gsc.tab=0&gsc.q=road%20accident%20Bangladesh&gsc.sort=&gsc.page="+str(x)+"&gsc.ref=more%3Awww.thedailystar.net" for x in range(1,11)]
dailystar_list = []
index = 0

for url in urls:
  wd.get(url)

  soup = BeautifulSoup(wd.page_source)

  for link in soup.find_all('a', {"class": "gs-title"}, href=True):
    dailystar_list.append(link['href'])
  
  
  time.sleep(2)

dailystar_list = list(set(dailystar_list))
dailystar_list

['https://www.thedailystar.net/backpage/news/12-killed-road-railway-accidents-1718203',
 'https://www.thedailystar.net/frontpage/bangladesh-road-accidents-in-2019-stats-by-nirapad-sarak-chai-1849588',
 'https://www.thedailystar.net/bangladesh-road-accidents-kill-211-in-30-days-shutdown-1899040',
 'https://www.thedailystar.net/country/road-accidents-in-bangladesh-during-eid-ul-fitr-2019-298-killed-1757374',
 'https://www.thedailystar.net/country/road-accident-in-bangladesh-3-killed-bus-hits-tree-in-bagerhat-1676374',
 'https://www.thedailystar.net/opinion/interviews/news/compensation-road-crash-victims-means-accountability-1843708',
 'https://www.thedailystar.net/country/bangladesh-road-accidents-in-2019-saw-significant-rise-1849381',
 'https://www.thedailystar.net/country/2-bangladeshi-youth-killed-america-georgia-road-accident-1449430',
 'https://www.thedailystar.net/sports/bangladesh-cricket/abdur-razzak-and-his-family-suffer-injury-road-accident-1425478',
 'https://www.thedailystar.

In [6]:
df_dict = {'Date_Published':[], 'Title':[], 'Text':[]}

for item in dailystar_list:
    r = requests.get(item)
    soup = BeautifulSoup(r.text)
    
    time.sleep(1)

    headings = soup.find_all('h1', attrs={'itemprop':'headline'})
    if len(headings) ==0:
        continue
    elif len(headings) >1:
        warnings.warn('More than one heading')
        print(headings)
    else:
        df_dict['Title'].append(headings[0].text)
    
    df_dict['Date_Published'].append(dateparser.parse(soup.find('meta', attrs={'itemprop':'datePublished'})['content']).strftime('%c %Z'))

    m_text = ''
    for para in soup.find_all('p', attrs={'class':None}):
        m_text = m_text + para.get_text() + '\n'

    df_dict['Text'].append(m_text.strip())

df = pd.DataFrame(df_dict)
df['Newspaper']='The Daily Star'
df

,Date_Published,Title,Text,Newspaper
0,Thu Mar 21 00:00:00 2019 UTC\+06:00,"13 killed in road, railway accidents",At least 13 people have been killed and 34 oth...,The Daily Star
1,Sun Jan 5 00:00:00 2020 UTC\+06:00,Monitoring lapses and indifference to blame,The number of road crashes and deaths rose ala...,The Daily Star
2,Sun May 3 17:15:46 2020 UTC\+06:00,Road accidents kill 211 in 30 days of shutdown...,At least 211 people were killed in road accide...,The Daily Star
3,Sat Jun 15 13:36:51 2019 UTC\+06:00,298 killed in accidents during Eid: Jatri Kaly...,At least 273 people were killed and 849 injure...,The Daily Star
4,Fri Dec 21 00:00:00 2018 UTC\+06:00,Three killed as bus hits roadside tree in Bage...,Three people were killed and around 20 others ...,The Daily Star
...,...,...,...,...
83,Sun May 27 17:22:04 2018 UTC\+06:00,2 Bangladeshi peacekeepers in Africa killed in...,Two soldiers of Bangladesh Army working in the...,The Daily Star
84,Mon Sep 21 19:07:29 2020 UTC\+06:00,Top PIB official dies in road accident in Jatr...,A top official of the Press Institute Banglade...,The Daily Star
85,Wed May 11 13:13:13 2016 UTC\+06:00,IU professor among 5 killed in road crashes,Five people including a teacher of Kushtia’s I...,The Daily Star
86,Mon Apr 5 16:29:10 2021 UTC\+06:00,513 people killed in 409 accidents last month:...,At least 513 people were killed and 598 were i...,The Daily Star


## Shortcomings

1. Not all results obtained were news articles.

2. Many results included Bangladeshis involved in road accidents abroad.

3. I could not extract all the relevant columns.